In [ ]:
%ls -all

In [ ]:
from importlib import import_module as _import
from timeit import default_timer as _timer

class Operator():
    """Node of directed graph representing an operator."""
    
    def __init__(self,
                 # Required inputs
                 id_: str,
                 name: str,
                 module_path: str,
                 # Optional inputs
                 label: str = None,
                 module_is_file: bool = False,
                 class_name: str = None,
                 class_params: dict = None,
                 operator_is_class: bool = False,
                 input_deps: dict = None,
                 input_params: dict = None,
                 output: dict = None,
                 operator_: object = None
                ):
        """Constructor.
        
        Args:
            id_: Unique graph-wide operator identifier,
                 which is usually set by the Controller
                 that assures uniqueness.
        
        Returns:
            Dictionary of operator execution profile
        """
        # Private vars
        self._me = "Operator():"
        self._id = id_
        self._name = name
        self._module_path = module_path
        self._run_profile = {}
        
        # Load new operator
        if operator_ is None:

            # Contained in package
            if not module_is_file:
                
                if class_name is None:
                    
                    # Access directly to operator with module
                    # 'path' and 'name' of operator, which can
                    # either be a method or a class. The latter
                    # case is determined at execution with the
                    # 'operator_is_class' flag.
                    try:
                        operator = getattr(_import(module_path), name)

                    except (ImportError, AttributeError) as err:
                        raise ImportError("{} Failed loading operator '{}.{}'."\
                                          .format(self._me, module_path, name))
                
                elif class_params is None:
                    
                    # Access to operator via class initialization
                    # without parameters passed to the constructor.
                    try:
                        operator = getattr(getattr(_import(module_path), class_name)(), name)
                    
                    except (ImportError, AttributeError) as err:
                        raise ImportError("{} Failed loading operator '{}.{}().{}'."\
                                          .format(self._me, module_path, class_name, name))
                
                else:
                    
                    # Access to operator via class initialization
                    # with parameters passed to the constructor.
                    try:
                        operator = getattr(getattr(_import(module_path), class_name)(**class_params), name)
                    
                    except (ImportError, AttributeError) as err:
                        raise ImportError("{} Failed loading operator '{}.{}(**class_params).{}'."\
                                          .format(self._me, module_path, class_name, name))
        
            # Contained in file
            else:
                
                # TODO: 
                print("TODO: load module with given name of Python file.")
        
        # Run operator with no return
        try:
            if output is None:
                _ = self._run(operator,
                              input_deps,
                              input_params,
                              operator_is_class)
            elif len(output) == 1:
                output[next(iter(output))] = self._run(operator,
                                                       input_deps,
                                                       input_params,
                                                       operator_is_class)
            else:
                raise ValueError("Shared 'output' object is expected "+
                                 "to hold only 1 output")
            
            # Record success
            self._run_profile["success"] = True
        
        except ValueError as err:
            print(err)
            self._run_profile["success"] = False
            pass
        
        except Exception as err:
            print(err)
            self._run_profile["success"] = False
            pass
            
    @property
    def id(self) -> str:
        return self._id
    
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def module_path(self) -> str:
        return self._module_path
    
    @property
    def run_profile(self) -> dict:
        return self._run_profile
    
    def _run(self,
             operator: object,
             input_deps: dict,
             input_params: dict,
             operator_is_class: bool):
        
        if input_deps is not None:
            if input_params is not None:
                
                # Sanity check for duplicates (TODO: check this!!)
                if input_deps.keys() in input_params.keys():
                    raise ValueError("{} Duplicate operator arguments between 'input_deps' "+
                                     "and 'input_params' detected. Check your settings."\
                                     .format(self._me))
                
                # Operator uses both mutable input objects,
                # which it can modify, and static parameters
                return self._profiled_run(operator, **input_deps, **input_params)[0]
            
            else:
                
                # Operator uses only mutable input objects,
                # which it can modify
                return self._profiled_run(operator, **input_deps)[0]
            
        elif input_params is not None:
            
            # Operator uses only static parameters
            return self._profiled_run(operator, **input_params)[0]
        
        else:
            
            # Operator needs no arguments
            return self._profiled_run(operator)[0]
    
    def _profiled_run(self, operator, *args):
        
        # Capture time
        time_start = _timer()
        
        # Run and measure
        return (
            operator(*args),
            self._runtime_ms(time_start)
        )
    
    def _runtime_ms(self, time_start):
        self._run_profile["runtime_ms"] = (_timer() - time_start) * 1000
        return _
        

In [ ]:
Operator(id_="t",
         name="method_no_args_with_return",
         module_path="testlib.test_app").run_profile

In [3]:
data = dict()

In [4]:
data["blockA"] = 3

In [6]:
data["blockB"] = { "blockB": 10 }

In [7]:
data

{'blockA': 3, 'blockB': {'blockB': 10}}

In [8]:
def change_me(sample: dict):
    sample["blockB"] = 11

In [9]:
change_me(data["blockB"])

In [10]:
data

{'blockA': 3, 'blockB': {'blockB': 11}}

In [11]:
data

{'blockA': 3, 'blockB': {'blockB': 11}}

In [12]:
copy_of_data = data

In [13]:
copy_of_data["blockA"] = 5

In [14]:
copy_of_data

{'blockA': 5, 'blockB': {'blockB': 11}}

In [15]:
data

{'blockA': 5, 'blockB': {'blockB': 11}}

In [16]:
from timeit import default_timer as _timer

In [17]:
_timer()

172241.0760553

In [18]:
type(_timer())

float